In [70]:
import itertools
import torch
import pickle
import numpy as np
import ZH_Nakamura
import subprocess
import os
torch.set_default_dtype(torch.float64)
import sys
sys.path.append('..')
from Tools import syncer 
from Tools import user
from Tools import helpers
import ROOT

In [71]:
def make_weight_ratio(r, weights, base_points):
    weight_ratio=0
    nCoefficients=len(r.coefficient_names)
    row,column=np.triu_indices(nCoefficients+1)
    i, j = np.triu_indices(8)                                                                                       
    for i_comb, combination in enumerate(r.combination_list):
        weight_ratio+=torch.outer(torch.from_numpy(weights[r.combination_list[i_comb]]/weights[()]),(base_points[:,row[i_comb]]*base_points[:,column[i_comb]]))
    return weight_ratio

In [72]:
def f_loss(r,features,base_points,weights):
        fhat = 1./(1. + r.predict_r_hat(features,base_points))
        weight_ratios=make_weight_ratio(r, weights, base_points)
        loss = (torch.from_numpy(weights[()])*torch.transpose(weight_ratios*fhat**2 + (1-fhat)**2,0,1))
        return loss

In [73]:
class R:
    def __init__(self,nfeatures,coefficient_names):
        self.nfeatures         = nfeatures
        self.coefficient_names = coefficient_names
        self.combination_list=list(itertools.chain.from_iterable(itertools.combinations_with_replacement(self.coefficient_names, i) for i in np.arange(0,3)))
        self.n_hat = {combination: self.make_NN() for combination in self.combination_list}
        
    def make_NN(self, hidden_layers  = [32, 32, 32, 32]):
        '''
        Creates the Neural Network Architecture
        '''
        model_nn = [torch.nn.BatchNorm1d(self.nfeatures), torch.nn.ReLU(), torch.nn.Linear(self.nfeatures, hidden_layers[0])]
        for i_layer, layer in enumerate(hidden_layers):
            model_nn.append(torch.nn.Linear(hidden_layers[i_layer], hidden_layers[i_layer+1] if i_layer+1<len(hidden_layers) else 1))
            if i_layer+1<len(hidden_layers):
                model_nn.append( torch.nn.ReLU() )
        return torch.nn.Sequential(*model_nn)

    def evaluate_NN(self, features):
        '''Evaluate Neural Network: The zeroth dimension of features is the number of data points and and the first dimension
        is the number of features(variables). Returns the output of the NNs of dimensions: (noutput,ndatapoints)
        '''
        noutputs=len(self.combination_list)
        ndatapoints=features.shape[0]
        
        output=torch.zeros((noutputs,ndatapoints))
        for i in range(noutputs):
            x=self.n_hat[self.combination_list[i]](features)
            if i==0:
                output[i,:]=1
            else:
                output[i,:]=torch.flatten(x)            
        return output
        
    
    def predict_r_hat(self, features, base_points):
        '''
        Evaluate positive xsec ratio for given theta and.
        First it fills the coefficients of the matrix containing the upper cholesky decomposition.
        Then it computes the multiplication of this matrix with the basepoints and squares it and sums it.
        '''
        ndatapoints=features.shape[0]
        output_NN = self.evaluate_NN(features)
        n_terms=len(self.coefficient_names)
        row,column=np.triu_indices(n_terms+1)
        Omega=torch.zeros((n_terms+1,n_terms+1,ndatapoints))
        for i in range(0, len(row)):
            Omega[row[i]][column[i]][:]=output_NN[i,:]
        Omega_swapped=torch.swapaxes(Omega,1,2)
        Omega_swapped=torch.swapaxes(Omega_swapped,0,1)
        
        
        out=torch.matmul(Omega_swapped,torch.transpose(base_points,0,1))
        #print("Shape of Omega")
        #print(Omega_swapped.shape)
        #print("Shape of basepoints")
        #print(torch.transpose(base_points,0,1).shape)
        #print("Shape of out")
        #print(out.shape)
        return torch.linalg.norm(out, 2, 1)
    
    def save(self,fileName):
        outfile = open(fileName,'wb')
        pickle.dump(self, outfile)
        outfile.close()
        
    @classmethod
    def load(self, fileName):
        infile = open(fileName,'rb')
        print(fileName)
        new_dict = pickle.load(infile)
        infile.close()
        return new_dict

In [74]:
n_features=6
#coefficients=['cHQ3', 'cHW', 'cHWtil']
#base_points = np.asarray([np.array([1, value1, value2, value3]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5] for value3 in [-1.5, -.8, .2, 0, .2, .8, 1.5]])
#coefficients=['cHW', 'cHWtil']
#base_points = np.asarray([np.array([1, value1, value2]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]  for value2 in [-1.5, -.8, .2, 0, .2, .8, 1.5]])
coefficients=['cHQ3']
base_points = np.asarray([np.array([1, value1]) for value1 in [-1.5, -.8, .2, 0., .2, .8, 1.5]])
print(base_points.shape)
r_NN = R(n_features, coefficients)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

(7, 2)


In [75]:
plot_directory="v4_Class_7"

nEvents=30000
learning_rate = 1e-3
device        = 'cuda' if torch.cuda.is_available() else 'cpu'
n_epoch       = 10000
plot_every    = 100

In [84]:
r=r_NN.predict_r_hat(features, torch.from_numpy(base_points))

In [85]:
r.shape

torch.Size([19010, 7])

In [88]:
r<0

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [77]:
# training data
import ZH_Nakamura 
ZH_Nakamura.feature_names = ZH_Nakamura.feature_names[0:6] # restrict features
features   = ZH_Nakamura.getEvents(nEvents)[:,0:6]
feature_names  = ZH_Nakamura.feature_names
plot_options   = ZH_Nakamura.plot_options
plot_vars      = ZH_Nakamura.feature_names

mask       = (features[:,feature_names.index('pT')]<900) & (features[:,feature_names.index('sqrt_s_hat')]<1800) 
features = features[mask]

n_features = len(features[0])
weights    = ZH_Nakamura.getWeights(features, ZH_Nakamura.make_eft())


WC='cHQ3'

pT=features[:,feature_names.index('pT')]
features=torch.from_numpy(features)
#w0_train       = torch.from_numpy(weights[()]).float().to(device)
#wp_train       = torch.from_numpy(weights[(WC,)]).float().to(device)
#wpp_train      = torch.from_numpy(weights[(WC,WC)]).float().to(device)
w0_train       = weights[()]
wp_train       = weights[(WC,)]
wpp_train      = weights[(WC,WC)]




Requested 30000 events. Simulated 30000 events and 30000 survive pT_min cut of 0.


In [78]:
all_params=[]
for comb in r_NN.combination_list:
        all_params+=r_NN.n_hat[comb].parameters()

In [79]:
#optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(list(all_params), lr=learning_rate)
losses = []
losses_train=[]
for comb in r_NN.combination_list:
    r_NN.n_hat[comb].train()

In [80]:
tex = ROOT.TLatex()
tex.SetNDC()
tex.SetTextSize(0.04)

In [61]:
r_NN.predict_r_hat(features, torch.from_numpy(base_points))

tensor([[1.0082, 1.0036, 0.9994,  ..., 0.9994, 0.9983, 0.9984],
        [1.0114, 1.0052, 0.9990,  ..., 0.9990, 0.9968, 0.9956],
        [0.9892, 0.9936, 1.0018,  ..., 1.0018, 1.0078, 1.0157],
        ...,
        [1.0112, 1.0050, 0.9991,  ..., 0.9991, 0.9971, 0.9963],
        [1.0130, 1.0057, 0.9990,  ..., 0.9990, 0.9971, 0.9970],
        [0.9978, 0.9984, 1.0005,  ..., 1.0005, 1.0024, 1.0052]],
       grad_fn=<CopyBackwards>)

In [83]:
f_loss(r_NN,features,torch.from_numpy(base_points),weights)

Loss shape
torch.Size([7, 19010])


tensor([[7.7912e-06, 5.5027e-06, 4.3895e-05,  ..., 1.7621e-05, 3.1295e-05,
         9.8619e-06],
        [2.3083e-06, 1.6291e-06, 1.2907e-05,  ..., 5.1603e-06, 8.7471e-06,
         2.9013e-06],
        [1.9521e-07, 1.3686e-07, 1.2452e-06,  ..., 4.8517e-07, 3.0469e-06,
         2.4382e-07],
        ...,
        [1.9521e-07, 1.3686e-07, 1.2452e-06,  ..., 4.8517e-07, 3.0469e-06,
         2.4382e-07],
        [2.8466e-06, 2.0105e-06, 1.6761e-05,  ..., 6.5564e-06, 1.7706e-05,
         3.5297e-06],
        [1.0317e-05, 7.3117e-06, 6.0147e-05,  ..., 2.3414e-05, 5.5019e-05,
         1.2690e-05]], grad_fn=<MulBackward0>)

In [14]:
features[:1]

tensor([[1.7764e+03, 2.6768e+02, 1.9520e+00, 9.5277e-01, 9.5967e-01, 3.8580e-01]])

In [11]:
for epoch in range(n_epoch):
    loss = f_loss(r_NN,features,torch.from_numpy(base_points),weights)
    print("epoch", epoch, "loss",  loss.item())
    
    losses.append(loss.item())
    optimizer.zero_grad()    
    loss.backward()
    optimizer.step()
    
    

epoch 0 loss 2.1847244508007053
epoch 1 loss 2.169864224994247
epoch 2 loss 2.155270877952321
epoch 3 loss 2.1410642890596536
epoch 4 loss 2.1272928402411604
epoch 5 loss 2.113877587875944
epoch 6 loss 2.1007287600482205
epoch 7 loss 2.08789100135243
epoch 8 loss 2.075437314911634
epoch 9 loss 2.0632288735071134
epoch 10 loss 2.0510301331069156
epoch 11 loss 2.038689172288373
epoch 12 loss 2.026096972542505
epoch 13 loss 2.0132002836166185
epoch 14 loss 1.9998616834567644
epoch 15 loss 1.9859331112158018
epoch 16 loss 1.9712063149731005
epoch 17 loss 1.9553850752680084
epoch 18 loss 1.9381403268198683
epoch 19 loss 1.9191538799301786
epoch 20 loss 1.8981321561264157
epoch 21 loss 1.8748648879061816
epoch 22 loss 1.8491650929937893
epoch 23 loss 1.8209206959698232
epoch 24 loss 1.7900898015577074
epoch 25 loss 1.7566710430901362
epoch 26 loss 1.7206958105548957
epoch 27 loss 1.682296470855106
epoch 28 loss 1.6416785426921703
epoch 29 loss 1.5990676106853594
epoch 30 loss 1.5547636484471

KeyboardInterrupt: 